In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install gensim

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
%cd "/content/drive/Shareddrives/SIADS - 697 Capstone Team Drive"
!ls -l

/content/drive/Shareddrives/SIADS - 697 Capstone Team Drive
total 593
-rw------- 1 root root    143 Jun 21 01:22 '697 Capstone: Team Ideas.gdoc'
-rw------- 1 root root  55216 Jul 30 00:01  arquitecture.xlsx
-rw------- 1 root root    143 Aug  2 23:35 'Capstone Project Deliverables.gdoc'
-rw------- 1 root root 498822 Jul 10 16:00 'Chat bot flows - Frame 6.jpg'
drwx------ 2 root root   4096 Jul  3 03:37  datasets
drwx------ 2 root root   4096 Jul 24 16:50  index
-rw------- 1 root root    143 Jun 28 17:00  Infrastructure.gdraw
-rw------- 1 root root    143 Jun 28 16:09 'Initial Capstone Project Proposal.gdoc'
drwx------ 2 root root   4096 Jul 20 23:17  models
drwx------ 2 root root   4096 Aug  8 20:23  new_package
drwx------ 2 root root   4096 Jul  3 17:07  NLP
drwx------ 2 root root   4096 Jul  3 03:29  notebooks
drwx------ 2 root root   4096 Jul  5 23:09  Notes
-rw------- 1 root root  17134 Jul 17 16:24  project_tasks_breakup.xlsx
drwx------ 2 root root   4096 Jul 11 02:15  Standups
-rw-

In [6]:
import re
import os
import pandas as pd
from gensim.parsing.preprocessing import preprocess_string, STOPWORDS
from bs4 import BeautifulSoup as BSHTML
from gensim import corpora, models, similarities
from gensim.similarities import Similarity
import pickle

In [ ]:
df = pd.read_json(
    'datasets/cleaned_data.json',
)

In [ ]:
df.head()

,asin,original_category,category_list,category_string,main_cat,title,4_features_combined,5_features_combined,4_features_tokenized,5_features_tokenized,category_group_count
0,0011300000,"[Electronics, Camera &amp; Photo, Video Survei...","[Electronics, Camera & Photo, Video Surveillan...","Electronics, Camera & Photo, Video Surveillanc...",Camera & Photo,Genuine Geovision 1 Channel 3rd Party NVR IP S...,Genuine Geovision 1 Channel 3rd Party NVR IP S...,Genuine Geovision 1 Channel 3rd Party NVR IP S...,"[genuin, geovis, channel, parti, nvr, softwar,...","[genuin, geovis, channel, parti, nvr, softwar,...",1392
1,0043396828,"[Electronics, Camera &amp; Photo]","[Electronics, Camera & Photo]","Electronics, Camera & Photo",Camera & Photo,"Books ""Handbook of Astronomical Image Processi...","Books ""Handbook of Astronomical Image Processi...","Books ""Handbook of Astronomical Image Processi...","[book, handbook, astronom, imag, process, rom,...","[book, handbook, astronom, imag, process, rom,...",5779
2,0060009810,"[Electronics, eBook Readers &amp; Accessories,...","[Electronics, eBook Readers & Accessories, eBo...","Electronics, eBook Readers & Accessories, eBoo...",Books,One Hot Summer,"One Hot Summer Electronics, eBook Readers & Ac...","One Hot Summer Electronics, eBook Readers & Ac...","[hot, summer, electron, ebook, reader, accesso...","[hot, summer, electron, ebook, reader, accesso...",399
3,0060219602,"[Electronics, eBook Readers & Accessories, eBo...","[Electronics, eBook Readers & Accessories, eBo...","Electronics, eBook Readers & Accessories, eBoo...",Books,Hurray for Hattie Rabbit: Story and pictures (...,Hurray for Hattie Rabbit: Story and pictures (...,Hurray for Hattie Rabbit: Story and pictures (...,"[hurrai, hatti, rabbit, stori, pictur, earli, ...","[hurrai, hatti, rabbit, stori, pictur, earli, ...",399
4,0060786817,"[Electronics, eBook Readers & Accessories, eBo...","[Electronics, eBook Readers & Accessories, eBo...","Electronics, eBook Readers & Accessories, eBoo...",Books,sex.lies.murder.fame.: A Novel,"sex.lies.murder.fame.: A Novel Electronics, eB...","sex.lies.murder.fame.: A Novel Electronics, eB...","[sex, li, murder, fame, novel, electron, ebook...","[sex, li, murder, fame, novel, electron, ebook...",399


### Train LSI Model

In [7]:
def create_folders_to_save_to_file(file):
    filepath = "/".join(file.split("/")[:-1])
    if filepath:
        os.makedirs(filepath, exist_ok=True)

def train_lsi(df, feature: str, save_model: str="../models/lsi_model.index"):
    """
    params:
    - df: Pandas DataFrame
    - feature: str value of the dataframe's column
    - save_model: 
    """
    dictionary = corpora.Dictionary(df[feature])
    corpus = [dictionary.doc2bow(text) for text in df[feature]]
    # word_counts = [[(dictionary[id], count) for id, count in line] for line in corpus]
    lsi = models.LsiModel(corpus, id2word=dictionary, num_topics=10)

    #index = similarities.MatrixSimilarity(lsi[corpus])  # transform corpus to LSI space and index it
    create_folders_to_save_to_file(save_model)
    create_folders_to_save_to_file('../lsi_dictionary/lsi_dict.pickle')
    create_folders_to_save_to_file('../lsi_model/lsi_model.pickle')
    index = Similarity(save_model, lsi[corpus], num_features=len(dictionary))  # transform corpus to LSI space and index it
    dictionary.save('../lsi_dictionary/lsi_dict.pickle')
    lsi.save('../lsi_model/lsi_model.pickle')
    index.save(save_model)
    return lsi, index, dictionary

In [ ]:
# Create model
lsi, index, dictionary = train_lsi(df, "4_features_tokenized", save_model="../models/lsi_model.index")

### Load LSI Model



In [52]:
%cd "/content/drive/Shareddrives/SIADS - 697 Capstone Team Drive/index/lsi_models"

[Errno 2] No such file or directory: '/content/drive/Shareddrives/SIADS - 697 Capstone Team Drive/index/lsi_models'
/content/drive/Shareddrives/SIADS - 697 Capstone Team Drive/index/lsi_models


In [53]:
from gensim.models import LsiModel
from gensim.similarities import Similarity
from gensim.corpora.dictionary import Dictionary

lsi = LsiModel.load('../../models/lsi_model/lsi_model.pickle')
index = Similarity.load('../../index/models/lsi_model.index')
dictionary = Dictionary.load('../../datasets/m1_data/lsi_dictionary/lsi_dict.pickle')

In [54]:
print(dictionary)

Dictionary(137843 unique tokens: ['camera', 'channel', 'dongl', 'dvr', 'electron']...)


### Model Inference

In [59]:
query_term = "camera dvr"
# query_term = "computer speakers with microphone"
# query_term = "i want a kingston sd card"
# query_term = "i want a computer monitor"
vec_bow = dictionary.doc2bow(query_term.lower().split()) # TFIDF
print("Vec bow", vec_bow)
vec_lsi = lsi[vec_bow]  # convert the query to LSI space # TruncatedSVD
print("Vec LSI Space", vec_lsi)
sims = index[vec_lsi]  # perform a similarity query against the corpus
# print(list(enumerate(sims)))  # print (document_number, document_similarity) 2-tuples
# 
sims = sorted(enumerate(sims), key=lambda item: -item[1])
print(sims[:10])
# for doc_position, doc_score in sims[:10]:
#     print(doc_score, df.category_list[doc_position])
#     print(df.title[doc_position])
#     print("======================")

Vec bow [(0, 1), (3, 1)]
Vec LSI Space [(0, 0.2039341602397799), (1, 0.7621092474756763), (2, 0.14032488284642844), (3, 0.09197423293906985), (4, 0.019777977557260223), (5, 0.10325393998406172), (6, 0.1298077648166184), (7, 0.02207169594315285), (8, 0.0627457074205551), (9, 0.002934396214632552)]


/usr/local/lib/python3.7/dist-packages/gensim/similarities/docsim.py:518: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  result = numpy.hstack(shard_results)


[(773116, 0.9971143), (773123, 0.99705184), (533068, 0.9956884), (28134, 0.9953094), (58502, 0.9953094), (199160, 0.9949553), (680836, 0.99494034), (234819, 0.99483955), (227822, 0.9948117), (735669, 0.9947904)]
